# Batch (parallel) Demand Forecasting using PyTorch Forecasting


**Batch training, tuning, and deploying** are common tasks in machine learning. They require training simple models, on data batches, typcially corresponding to different locations, products, etc. Batch training can take less time to process all the data at once, but only if those batches can run in parallel!

This notebook showcases how to conduct batch forecasting using the <a href="https://github.com/ray-project/ray_lightning">Ray Lightning plugin</a> (which runs on top <a href="https://docs.ray.io">Ray</a>) to speed up training and inference of Google's <a href="https://github.com/google-research/google-research/tree/master/tft">TemporalFusionTransformer</a> algorithm for RNN with LSTM, which has been adapted by <a href="https://pytorch-forecasting.readthedocs.io">PyTorch Forecasting</a>, which in turn is built on <a href="https://pytorch-lightning.readthedocs.io">PyTorch Lightning</a>. PyTorch Lightning is a set of APIs to simplify PyTorch, similar to the relationship of Keras to TensorFlow.

<img src="../../../../AnyscaleDemos/ray_air_demos/images/embarrassingly_parallel.png" style="width: 50%"/>

For the data, we will use the [NYC Taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). This popular tabular dataset contains historical taxi pickups by timestamp and location in NYC.

For the training, we will train a separate forecasting model to predict #pickups at each location in NYC at hourly level for the next week. Specifically, we will train multiple TemporalFusionTransformer models using different hyperparameters, on different segments of data, to find the best model per segment of data.  For convenience, we have already clustered the `pickup_location_id` column in the dataset into 2 groups (or clusters of data).

**Demo notes** <br>
See [README](https://github.com/christy/AnyscaleDemos/tree/main/forecasting_demos#-setup-instructions-for-anyscale) for instructions how to run this notebook on Anyscale.

# Contents

In this this tutorial, you will learn how to:
 1. [Define how to load and prepare Parquet data](#prepare_data) 
 2. [Define a train and evaluate functions](#define_trainable)
 3. [Run batch training and inference with Ray Tune](#run_tune_search)
 4. [Load a model from checkpoint](#load_checkpoint)
 5. [Create a validation forecast from restored checkpoint model](#create_inference)
 6. [Deploy a model from checkpoint using Ray Serve](#deploy_model)


# Walkthrough

```{tip}
Prerequisite for this notebook: Read the [Key Concepts](https://docs.ray.io/en/latest/tune/key-concepts.html) page for Ray Tune.
```

Let us start by importing a few required libraries, including open-source [Ray](https://github.com/ray-project/ray) itself!

In [1]:
!python --version
import os, random
import typing
num_cpu = os.cpu_count()

print(f"Number of CPUs in this system: {num_cpu}")
from typing import Tuple, List, Union, Optional, Callable
from datetime import datetime, timedelta
import time
import pandas as pd
import numpy as np

print(f"numpy: {np.__version__}")
import matplotlib.pyplot as plt
import pickle

%matplotlib inline
import pyarrow
import pyarrow.parquet as pq
import pyarrow.dataset as pds

print(f"pyarrow: {pyarrow.__version__}")


Python 3.8.15
Number of CPUs in this system: 8
numpy: 1.22.3
pyarrow: 10.0.0


In [2]:
# import GPUtil #GPU status from NVIDA GPUs
# len(GPUtil.getGPUs())

In [9]:
import ray

if ray.is_initialized():
    ray.shutdown()
ray.init()

In [4]:
print(ray.cluster_resources())

{'node:127.0.0.1': 1.0, 'memory': 7033421824.0, 'CPU': 8.0, 'object_store_memory': 2147483648.0}


In [5]:
# Import forecasting libraries.
import torch 
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
#PyTorch Forecasting convenience APIs for PyTorch Lightning
import pytorch_forecasting as ptf
pl.seed_everything(415)  # Set global random seed
# PyTorch visualization uses Tensorboard
import tensorboard as tb
# tf.io.gfile = tb.compat.tensorflow_stub.io.gfile  #compatibility for PyTorch

print(f"torch: {torch.__version__}")
print(f"PyTorch Lightning: {pl.__version__}")
print(f"pytorch forecasting: {ptf.__version__}")
print(f"tensorboard: {tb.__version__}")
# Import ray libraries.
import ray_lightning
from ray_lightning import RayStrategy
from ray_lightning.tune import get_tune_resources, TuneReportCheckpointCallback
from ray import air, tune
from ray.tune.schedulers import ASHAScheduler

Global seed set to 415


torch: 1.13.1
PyTorch Lightning: 1.6.5
pytorch forecasting: 0.10.3
tensorboard: 2.11.2


In [6]:
# For benchmarking purposes, we can print the times of various operations.
# In order to reduce clutter in the output, this is set to False by default.
PRINT_TIMES = False

def print_time(msg: str):
    if PRINT_TIMES:
        print(msg)

# To speed things up, we’ll only use a small subset of the full dataset consisting of two last months of 2019.
# You can choose to use the full dataset for 2018-2019 by setting the SMOKE_TEST variable to False.
SMOKE_TEST = False

## Define how to load and prepare Parquet data <a class="anchor" id="prepare_data"></a>

First, we need to load some data. Since the NYC Taxi dataset is fairly large, we will filter files first into a PyArrow dataset. And then in the next cell after, we will filter the data on read into a PyArrow table and convert that to a pandas dataframe.

```{tip}
Use PyArrow dataset and table for reading or writing large parquet files, since its native multithreaded C++ adapter is faster than pandas read_parquet, even using engine=pyarrow.
```

In [7]:
# Define some global variables.
TARGET = "trip_quantity"
FORECAST_LENGTH = 28
# TODO: Change this according to where your git files are located.
# data_dir = "/home/ray/christy-air/forecasting_demos/Ray_v2/data"
data_dir = "~/Documents/github_christy_latest/AnyscaleDemos/forecasting_demos/Ray_v2/data"
data_dir = os.path.expanduser(data_dir)
DATA_FILENAME = data_dir + "/clean_taxi_hourly_clustered.parquet"

# Use smoke testing or not.
# sample_clusters = ["erratic"] if SMOKE_TEST else ["smooth", "erratic"]
sample_clusters = ["erratic"]

# Display what data will be used.
print(f"data file: {DATA_FILENAME}")
print(f"Clusters: {sample_clusters}")

data file: /Users/christy/Documents/github_christy_latest/AnyscaleDemos/forecasting_demos/Ray_v2/data/clean_taxi_hourly_clustered.parquet
Clusters: ['smooth', 'erratic']


In [8]:
############
# STEP 1. Define Python functions to read and prepare data.
############

# Function to read a pyarrow.Table object using pyarrow parquet, pq
def read_data_from_clean(
    file: str, 
    sample_id: np.int32) -> pd.DataFrame:

    df = pq.read_table(
        file,
        filters=[
            ("pulocationid", "not in", ['264', '265', '199']),
            ("ts_type", "=", sample_id),
        ],
        columns=[
            "time_idx",
            "pulocationid",
            "day_hour",
            "trip_quantity",
            "ts_type",
        ],
    ).to_pandas()
    return df

def prepare_data_from_clean(
    file: str, 
    sample_cluster_id: str) -> pd.DataFrame:

    # Load data from clean sample.
    df = read_data_from_clean(file, sample_cluster_id)
    df.rename(columns={"pulocationid": "pickup_location_id"}, inplace=True)
    df['trip_quantity'] = df['trip_quantity'].astype(np.float32)

    # Abort Tune to avoid Tune Error if df has too few rows
    if df.shape[0] < FORECAST_LENGTH:
        print_time(
            f"Location {sample_cluster_id} has only {df.shape[0]} rows"
        )
        session.report(dict(error=None))
        return None
    else:
        return df


## Define a train and evaluate functions <a class="anchor" id="define_trainable"></a>

In [9]:
############
# STEP 2.  Define Python functions to train and evaluate a model.
############
# Convert pandas data to PyTorch tensors.
def convert_pandas_pytorch_timeseriesdata(
    input_data_pandas_df:pd.DataFrame, 
    config:dict
) -> typing.Union['pytorch_forecasting.data.timeseries.TimeSeriesDataSet',
                  'torch.utils.data.dataloader.DataLoader']:

    """Converts pandas dataframe into TimeSeries folded tensors following 
       the backtesting technique.  A generator for doing the folding is 
       per batch also created.  One for the training data.  
       Another for the validation data.  

    Inputs:
        pd.DataFrame: All the input data
        dict: config is a configuration file containing hard-coded settings.

    Returns:
        'pytorch_forecasting.data.timeseries.TimeSeriesDataSet': training data
        'torch.utils.data.dataloader.DataLoader': training data loader
        'torch.utils.data.dataloader.DataLoader': validation data loader
    """
    
    # specify data parameters
    FORECAST_HORIZON = config.get("forecast_horizon", 168)
    CONTEXT_LENGTH = config.get("context_length", 63)
    BATCH_SIZE = config.get("batch_size", 32)
    NUM_TRAINING_WORKERS = config.get("num_training_workers", 4)
    id_col_name = "pickup_location_id"
    target_value = "trip_quantity"
    
    df = input_data_pandas_df.copy()
    
    # define forecast horizon and training cutoff
    max_prediction_length = FORECAST_HORIZON  #decoder length = 1 week forecast horizon
    max_encoder_length = CONTEXT_LENGTH  # window or context length
    training_cutoff = df["time_idx"].max() - max_prediction_length 

    # convert pandas to PyTorch tensor
    training_data = ptf.data.TimeSeriesDataSet(
        df[lambda x: x.time_idx <= training_cutoff],
        allow_missing_timesteps=True,
        time_idx="time_idx",
        target=target_value,
        group_ids=[id_col_name],
        min_encoder_length=5,  # min 5 historical values must exist
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=max_prediction_length,
        static_categoricals=[id_col_name],
        # static_reals=["avg_population_2017", "avg_yearly_household_income_2017"],
        static_reals=[],
        time_varying_known_categoricals=["day_hour"],
        # group of categorical variables can be treated as one variable
        # variable_groups={"special_days": special_days},  
        time_varying_known_reals=["time_idx", ],
                            # "mean_item_loc_weekday",
                            # "binned_max_item"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=[target_value,],

        # https://pytorch-forecasting.readthedocs.io/en/v0.2.4/_modules/pytorch_forecasting/data.html
        target_normalizer=ptf.data.GroupNormalizer(
            groups=["pickup_location_id"], 
            transformation="softplus"  #forces positive values
        ), 
        add_relative_time_idx=True, # add as feature
        add_target_scales=True, # add avg target_value as feature
        add_encoder_length=True, # add as feature
    )
    
    # create PyTorch dataloader for training
    train_loader = training_data\
                        .to_dataloader(
                            train=True, 
                            batch_size=BATCH_SIZE, 
                            num_workers=NUM_TRAINING_WORKERS)
    
    # create validation PyTorch data 
    # (predict=True) means make do inference using the validation data
    val_dataset = ptf.data.TimeSeriesDataSet\
                    .from_dataset(
                        training_data, 
                        data=df, 
                        predict=True, 
                        stop_randomization=True)

    # create PyTorch dataloaders for inference on validation data
    validation_loader = val_dataset\
                    .to_dataloader(
                        train=False, 
                        batch_size=BATCH_SIZE * 10, 
                        num_workers=NUM_TRAINING_WORKERS)
    
    # return original df converted to PyTorch tensors, and pytorch loaders
    return training_data, train_loader, validation_loader

################
# calculate WQL, MAE
# note: to get a single item quantile prediction:
# example: quantile p50 for itemid="140"
# y_quantiles[1].detach().cpu()[43, : x["decoder_lengths"][43]] 
# raw predictions are a dictionary from which quantiles can be extracte
# TIP:  Use x to figure out mappings indexes to locations [141, 43, 144]
################
def evaluate_model(y_actual:'torch.Tensor', 
             y_quantiles:'torch.Tensor', 
             quantile_list:list)->'torch.Tensor':
    """Calculate weighted quantile loss given actuals, quantile predictions,
       and list of desired quantiles to average over.
    Inputs:
        'torch.Tensor': y_actual is a tensor of actual values 
        'torch.Tensor': y_quantiles is a tensor of quantile predictions
        'list': List of quantiles to average over

    Returns:
        'torch.Tensor': weighted quantile loss over all the desired quantiles
    """

    assert not y_actual.requires_grad
    
    all_losses = []
    for i, q in enumerate(quantile_list):
        sum_actuals = torch.sum(torch.abs(y_actual[i]))
        errors = torch.abs(y_actual[i] - y_quantiles[i][:, i])
        all_losses.append(
            torch.where(y_quantiles[i][:, i] > y_actual[i],
                        (1-q) * errors, 
                        q * errors ).unsqueeze(1))
        
        if torch.is_nonzero(sum_actuals):
            all_losses[i] = torch.sum(all_losses[i]).div(sum_actuals)
        else:
            all_losses[i] = torch.empty_like(all_losses[i])
            
        # Only for 50th percentile, calculate MAE
        if q == 0.5:
            MAE = torch.sum(errors)/y_actual.shape[0]
    
    WQL = torch.mean(torch.stack(all_losses), dim=0)
    MAE = torch.mean(MAE)
    return WQL, MAE

## Create and train a baseline model

In [10]:
%%time

# Config parameters for baseline model.
num_training_workers = min(num_cpu - 2, 32)
FORECAST_CONFIG = {"forecast_horizon": 168, "context_length": 63,
          "num_gpus":0, "batch_size": 128, 
          "num_training_workers": num_training_workers,
          "sample_cluster_id": ['erratic'],
         }

# Test reading data pandas from sample clean data.
df = prepare_data_from_clean(DATA_FILENAME, sample_clusters[1])
# Display what data will be used.
sample_locations = df.loc[(df.ts_type.isin(sample_clusters)), 'pickup_location_id'].unique()
print(df.head())

# convert data from pandas to PyTorch tensors
print(f"Input data type: {type(df)}")
train_dataset, train_loader, validation_loader = convert_pandas_pytorch_timeseriesdata(
    df, FORECAST_CONFIG
)
print(f"Converted data type: {type(train_dataset)}")

# calculate baseline mean absolute error, i.e. predict next value as the last available value from the history
actuals = torch.cat([y for x, (y, weight) in iter(validation_loader)])
baseline_predictions = ptf.models.Baseline().predict(validation_loader)

## EVALUATE THE BASELINE MODEL
# print MAE
baseline_error = (actuals - baseline_predictions).abs().mean()
print(f"baseline model MAE: {baseline_error}")

# Erratic cluster
# baseline model MAE: 71.43466186523438

# Smooth cluster
# baseline model MAE: 20.886438369750977


   time_idx pickup_location_id      day_hour  trip_quantity  ts_type
0      4371                 10      Monday_3            0.0  erratic
1      3926                 10  Wednesday_14            1.0  erratic
2      4773                 10  Wednesday_21            2.0  erratic
3      5046                 10      Monday_6            2.0  erratic
4      3911                 10    Tuesday_23            1.0  erratic
Input data type: <class 'pandas.core.frame.DataFrame'>
Converted data type: <class 'pytorch_forecasting.data.timeseries.TimeSeriesDataSet'>


/Users/christy/miniconda3/envs/dev38/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:261: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/Users/christy/miniconda3/envs/dev38/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:261: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(


baseline model MAE: 71.43466186523438
CPU times: user 1.1 s, sys: 242 ms, total: 1.34 s
Wall time: 6.44 s


## Run batch training and inference with Ray Tune <a class="anchor" id="run_tune_search"></a>

**Recall what we are doing, high level, is training several different models per cluster of pickup locations.** To do this, we need to be able to:
 - Pre-process all the data per model, distributing the runtime.
 - Run parallel trials, which compare different hyperparameter settings per model.
 - Inspect the results of the tuning experiment and deploy only the best model per cluster of pickup locations.

**Below are the general steps to preprocess data and automatically tune models from any combination of AI/ML Python libraries using distributed computing with [Ray AIR](https://docs.ray.io/en/latest/ray-air/getting-started.html) and [APIs](https://docs.ray.io/en/latest/ray-air/package-ref.html):**

<ol>
    <li>Define Python functions <b>`preprocess`</b> to read and prepare a segment of data.
    <li>Define Python functions to `<b>train`</b> and <b>`evaluate`</b> a model on a segment of data.
    <li>Define a calling function <b>`train_models`</b>, which calls all the above functions, and will be called in parallel for every permutation in the Tune search space!<br>
    Inside this <a href="https://docs.ray.io/en/latest/tune/api_docs/trainable.html#trainable-docs">`train_models`</a> function:
    <ul>
        <li>📖 The input parameters must include a config dictionary argument.
        <li>📈 The tuning metric (a model's loss or error) must be calculated and reported using session.report().
        <li>✔️ Checkpoint (save) the model is recommended for fault tolerance and easy deployment later.
    </ul>
    <li><b>Configure distributed compute scaling</b>.
    <li><b>Define a Ray Tune search space</b> of all training parameters.
    <li><b>Specify a hyperparameter search strategy.</b>  In this example, we use an <a href="https://blog.ml.cmu.edu/2018/12/12/massively-parallel-hyperparameter-optimization/">Asynchronous Hyperband scheduler</a>. This scheduler decides at each iteration which trials are likely to perform badly, and stops these trials. This way we don’t waste any resources on bad hyperparameter configurations.
    <li><b>Run the experiment</b> using Ray AIR APIs with Ray Lightning.
</ol>

Below is the additional code we would add.

In [ ]:
############
# STEP 3.  Define a calling function `train_models`, which calls all 
#          the above functions, and will be called in parallel for every 
#          permutation in the Tune search space.
############
def train_models(
    config: dict,
    strategy: 'ray_lightning.ray_ddp.RayStrategy',
    filename: str):
    """Define a calling function to read data, define a model and train it.

    Inputs:
        dict: configuration dictionary with hard-coded runtime values
        str:  data filename where to read a segment of data
    """
    # Get data segment ID from Tune search parameters
    sample_cluster_id = config.get("sample_cluster_id", ["erratic"])
    print_time(f"Data segment: {sample_cluster_id}")
    
    # Read a segment of data for multi-model training in parallel.
    df = prepare_data_from_clean(filename, sample_cluster_id)
        
    # Convert data from pandas to PyTorch Forecasting tensors.
    train_dataset, train_loader, validation_loader = \
        convert_pandas_pytorch_timeseriesdata(df, config)
    
    # Checkpoint a custom object.
    # PyTorch Lightning checkpointing not yet fully integrated with Ray AIR.
    with open('val_loader.pickle', 'wb') as handle:
        pickle.dump(validation_loader, handle)
    
    # Create your PTF model.
    model = ptf.models.TemporalFusionTransformer.from_dataset(
        train_dataset,
        learning_rate=config.get("lr", 0.01),
        hidden_size=config.get("hidden_size", 40), # num neurons in each layer, bigger runs more slowly
        # lstm_layers=HIDDEN_LAYERS, #LSTM layers=1 is default for tft architecture
        attention_head_size=config.get("attention_head_size", 4),  #default 4 cells in LSTM layer
        dropout=config.get("droupout", 0.1),
        hidden_continuous_size=config.get("hidden_continuous_size", 1),  #similar to categorical embedding size
        # 7 quantiles by default: [0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 0.98]
        # optimizer loss metric
        loss=ptf.metrics.QuantileLoss([0.25, 0.5, 0.75]),
        log_interval=10, #learning-rate logging every 10 batches
        reduce_on_plateau_patience=4, # reduce learning automatically
    )
    print(f"Number of parameters in network: {model.size()/1e3:.1f}k")
    
    # Configure early stopping.
    early_stop_callback = \
        pl.callbacks.EarlyStopping(monitor="val_loss",
                      min_delta=1e-4, 
                      patience=10, 
                      verbose=False, 
                      mode="min")
    
    # Configure logging.
    log_dir = os.getcwd() # + "/lightning_logs"
    logger = pl.loggers.TensorBoardLogger(save_dir=log_dir, name="", version="")
    
    # Create the Tune Reporting Callback.
    metrics = dict(loss="val_loss")
    callbacks = [
             early_stop_callback, 
             TuneReportCheckpointCallback(
                 metrics,
                 on="validation_end")]

    # Configure PyTorch trainer with Ray Lightning plugin.
    trainer = pl.Trainer(
        max_epochs=config.get("epochs", 30),
        gradient_clip_val=config.get("gradient_clip_val", 0.1),
        limit_train_batches=config.get("limit_train_batches", 30),  
        callbacks=callbacks,
        logger=logger,
        strategy=strategy, 
        # Suggest always use fast_mode, uses Tune instead of ptf logging.
        fast_dev_run=True,
    )
    trainer.fit(
        model,
        train_dataloaders=train_loader,
        val_dataloaders=validation_loader,
    )


In [ ]:
############
# STEP 4. Customize distributed compute scaling.
# https://docs.ray.io/en/master/train/config_guide.html
############
num_training_workers = min(num_cpu - 2, 32)
strategy = RayStrategy(num_workers=num_training_workers, 
                       num_cpus_per_worker=1, 
                       use_gpu=False,
                       find_unused_parameters=False,)

############
# STEP 5. Define a search space dict of all config parameters.
# https://docs.ray.io/en/latest/ray-air/tuner.html
############
FORECAST_CONFIG = {
    "forecast_horizon": 168,
    "context_length": 63,
    "num_gpus": 0,
    "num_training_workers": num_training_workers,
    "batch_size": 64,
    "epochs": 2,
    "lr": tune.grid_search([0.0005, 0.005, 0.05]),
    "hidden_size": 20,
    "dropout": 0.1,
    "hidden_continuous_size": 4,
    "attention_head_size": num_training_workers,
    "limit_train_batches": 1, 
    "sample_cluster_id": tune.grid_search(sample_clusters),
}
print(f"Data segment IDs: {sample_clusters}")

############
# Optional STEP 6. Specify the hyperparameter tuning search strategy.
# https://docs.ray.io/en/latest/tune/key-concepts.html#search-algorithms
############
# Select scheduler https://blog.ml.cmu.edu/2018/12/12/massively-parallel-hyperparameter-optimization/
scheduler = ASHAScheduler(
    max_t=FORECAST_CONFIG['epochs'],
    grace_period=1,
    reduction_factor=2)

<br>

💡 After you run the cell below, right-click on it and choose "Enable Scrolling for Outputs"! This will make it easier to view, since tuning output can be very long!

In [ ]:
############
# STEP 7. Run the experiment with Ray AIR APIs and Ray Lightning.
# https://docs.ray.io/en/latest/tune/examples/tune-pytorch-lightning.html
############
start = time.time()

# By default Callable function accepts only 1 input parameter called "config".
# Wrap Callable train function inside tune.with_parameters().
train_with_parameters = \
    tune.with_parameters(
        train_models,
        strategy=strategy,
        filename=DATA_FILENAME)

# Define a tuner object.
tuner = tune.Tuner(
        tune.with_resources(
            train_with_parameters,
            resources=get_tune_resources(num_workers=num_training_workers),
        ),
        tune_config=tune.TuneConfig(
            metric="loss",
            mode="min",
            scheduler=scheduler,
        ),
        run_config=air.RunConfig(
            # Redirect logs to relative path instead of default ~/ray_results/.
            local_dir="my_Tune_logs",
            # Specify name to make logs easier to find in log path.
            name="ptf_nyc",
        ),
        param_space=FORECAST_CONFIG,
    )

# Fit the tuner object.
results = tuner.fit()

total_time_taken = time.time() - start
print(f"Total number of models: {len(results)}")
print(f"Finished in: {(time.time()-start)/60} minutes")

# Total number of models: 6
# Finished in: 1.1374133070309957 minutes


Data segment IDs: ['smooth', 'erratic']


(train_model pid=2629) /Users/christy/miniconda3/envs/dev38/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:261: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
(train_model pid=2629)   rank_zero_warn(
(train_model pid=2629) /Users/christy/miniconda3/envs/dev38/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:261: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
(train_model pid=2629)   rank_zero_warn(
(train_model pid=2629) GPU available: False, used: False
(train_model pid=2629) TPU available: False, using: 0 TPU cores
(train_model pid=2629) IPU available: False, using: 0 IPUs
(train_model pid=2629) HPU available: False, using: 0 HPUs
(train_model pid=

(train_model pid=2629) Number of parameters in network: 32.4k


(RayExecutor pid=2633) /Users/christy/miniconda3/envs/dev38/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
(RayExecutor pid=2633)   new_rank_zero_deprecation(
(RayExecutor pid=2633) /Users/christy/miniconda3/envs/dev38/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: ParallelStrategy.torch_distributed_backend was deprecated in v1.6 and will be removed in v1.8.
(RayExecutor pid=2633)   return new_rank_zero_deprecation(*args, **kwargs)
(RayExecutor pid=2633) Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/6
(RayExecutor pid=2628) Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/6
(RayExecutor pid=2630) Initializing distributed: GLOBAL_RANK: 2, MEMBER: 3/6
(RayExecutor pid=2

Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_model_96e1b_00000,2023-01-22_22-38-49,,,ef376bdc89414d4eb44ded94af7260ba,christys-MacBook-Pro.local,,,127.0.0.1,2629,,,,,1674455929,,,,96e1b_00000,
train_model_96e1b_00001,2023-01-22_22-40-15,True,,fa8b83dbabd647a0a3a51dfb8c6a0e8f,christys-MacBook-Pro.local,1,18.34697723388672,127.0.0.1,2669,True,69.46443486213684,69.46443486213684,69.46443486213684,1674456015,0,,1,96e1b_00001,0.0016338825225830078
train_model_96e1b_00002,2023-01-22_22-41-44,True,,fa8b83dbabd647a0a3a51dfb8c6a0e8f,christys-MacBook-Pro.local,1,16.53215217590332,127.0.0.1,2669,True,68.1510100364685,68.1510100364685,68.1510100364685,1674456104,0,,1,96e1b_00002,0.0016338825225830078
train_model_96e1b_00003,2023-01-22_22-43-13,True,,fa8b83dbabd647a0a3a51dfb8c6a0e8f,christys-MacBook-Pro.local,1,37.89729690551758,127.0.0.1,2669,True,60.75404596328735,60.75404596328735,60.75404596328735,1674456193,0,,1,96e1b_00003,0.0016338825225830078


...

(train_model pid=2669) Number of parameters in network: 28.6k


(RayExecutor pid=3098) /Users/christy/miniconda3/envs/dev38/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
(RayExecutor pid=3098)   new_rank_zero_deprecation(
(RayExecutor pid=3098) /Users/christy/miniconda3/envs/dev38/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: ParallelStrategy.torch_distributed_backend was deprecated in v1.6 and will be removed in v1.8.
(RayExecutor pid=3098)   return new_rank_zero_deprecation(*args, **kwargs)
(RayExecutor pid=3098) Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/6
(RayExecutor pid=3101) Initializing distributed: GLOBAL_RANK: 3, MEMBER: 4/6
(RayExecutor pid=3102) Initializing distributed: GLOBAL_RANK: 4, MEMBER: 5/6
(RayExecutor pid=3

Epoch 0:  50%|█████     | 1/2 [00:15<00:15, 15.87s/it, loss=52.3, v_num=, train_loss_step=52.30]
Validation: 0it [00:00, ?it/s]0m 
(RayExecutor pid=3098) 
Validation DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]
(RayExecutor pid=3098) 
Epoch 0: 100%|██████████| 2/2 [00:49<00:00, 24.66s/it, loss=52.3, v_num=, train_loss_step=52.30, val_loss=37.90, train_loss_epoch=40.40]


(train_model pid=2669) /Users/christy/miniconda3/envs/dev38/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:261: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
(train_model pid=2669)   rank_zero_warn(
(train_model pid=2669) /Users/christy/miniconda3/envs/dev38/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:261: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
(train_model pid=2669)   rank_zero_warn(
(train_model pid=2669) GPU available: False, used: False
(train_model pid=2669) TPU available: False, using: 0 TPU cores
(train_model pid=2669) IPU available: False, using: 0 IPUs
(train_model pid=2669) HPU available: False, using: 0 HPUs
(train_model pid=

(train_model pid=2669) Number of parameters in network: 28.6k
(scheduler +33m29s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(scheduler +34m5s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(scheduler +34m40s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(scheduler +35m15s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer 

## Load a model from checkpoint  <a class="anchor" id="load_checkpoint"></a>

- After the Tune experiment has run, we can assemble the Tune ResultGrid object into a pandas dataframe.

- Next, we'll sort the pandas dataframe by data segment and error, and keep only the best model with minimum error per data segment.


In [ ]:
# [print(i.checkpoint) for i in results]
[print(i.config) for i in results]

In [ ]:
# get a list of training loss errors
loss = [i.metrics.get("loss", 10000.0) for i in results]

# get a list of model params
# batch_size = [i.config["batch_size"] for i in results]
lr = [i.config["lr"] for i in results]
dropout = [i.config["dropout"] for i in results]
data_segment = [i.config["sample_cluster_id"] for i in results]

# get a list of checkpoints
checkpoints = [i.checkpoint for i in results]

# Assemble a pandas dataframe from Tune results
results_df = pd.DataFrame(
    zip(loss, lr, dropout, data_segment, checkpoints),
    columns=["loss", "learning_rate", "droput", "data_segment", "checkpoint"],
)
print(results_df.dtypes)
print(results_df.head(8))

# Get model error for a specific model.
best_model_error = results_df.loc[(results_df.data_segment=="erratic"), 'loss'].min()
print("#########")
print(f"Baseline error cluster1: {baseline_error}")
print(f"Best tuned error cluster1: {best_model_error}")
print("#########")
print()

In [ ]:
# Get model path for a specific model.
model_path = results_df.iloc[5, -1:][0]  #Pick row num from df displayed above
model_path = str(model_path).split("local_path=")[1]
model_dir = str(model_path).split("checkpoint_000001")[0]
model_path = model_path.split(")")[0] + "/checkpoint"
print(f"model_dir: {model_dir}")
print(f"checkpoint_path: {model_path}")

# Restore a model from checkpoint.
best_model = ptf.models.TemporalFusionTransformer.load_from_checkpoint(model_path)
print(f"Restored model type: {type(best_model)}")

## Create a validation forecast from restored checkpoint model <a class="anchor" id="create_inference"></a>

Finally, we will restore the best models per cluster of data from checkpoint, perform inference on validation data, and inspect the "backtest" forecasts.

- We will easily obtain AIR Checkpoint objects from the Tune results. 
- We will restore a PyTorch Forecasting model directly from checkpoint, and demonstrate it can be used for prediction.

```{tip}
[Ray AIR Predictors](https://docs.ray.io/en/latest/ray-air/predictors.html) make batch inference easy since they have internal logic to parallelize the inference.
```


In [ ]:
# Restore a validation loader from pickle file.
filename = model_dir + "val_loader.pickle"
print(f"filename: {filename}")
with open(filename, 'rb') as handle:
    validation_loader = pickle.load(handle)
type(validation_loader)

In [ ]:
################
# calculate WQL, MAE
# note: to get a single item quantile prediction:
# example: quantile p50 for itemid="140"
# y_quantiles[1].detach().cpu()[43, : x["decoder_lengths"][43]] 
# raw predictions are a dictionary from which quantiles can be extracte
# TIP:  Use x to figure out mappings indexes to locations [141, 43, 144]
################
print(f"available quantiles: {best_model.loss.quantiles}")
raw_predictions, x = best_model.predict(validation_loader, mode="raw", return_x=True)
desired_quantiles = [0.25, 0.5, 0.75]
y_quantiles = best_model.to_quantiles(raw_predictions, desired_quantiles)
WQL, MAE = evaluate_model(actuals, y_quantiles, desired_quantiles)
print(f"Mean WQL over {desired_quantiles}: {WQL}")
print(f"Mean AE at p50: {MAE}")

In [ ]:
# Plot inference forecasts for some unique_ids.
# some_unique_ids = random.sample(range(0, pred.shape[0]), 10)
some_unique_ids = [25, 41, 14, 24, 4]
for idx in some_unique_ids:
    best_model.plot_prediction(x, raw_predictions, idx=idx, add_loss_to_title=True)

## Deploy a model from checkpoint using Ray Serve
<a class="anchor" id="deploy_model"></a>

Coding steps for Deployment:
- **Step 1**. Instantiate a batch predictor using Ray AIR checkpoints.
- **Step 2**. Create some test data.
- **Step 3**. Run `batch_predictor.predict(test_data)`.

Replace Step 3 above with these steps for a custom predictor:
- **Step 3**.  Define a Ray Serve deployment class by using a Ray decorator @serve.deployment.
- **Step 4**.  Deploy the predictor.
- **Step 5**.  Query the deployment and get the result.

Steps 3-5 above are only required if you are using a custom predictor (such as ARIMA, Prophet, or PyTorch Forecasting).  Otherwise for Ray AIR-integrated ML Libraries (HuggingFace transformers, PyTorch, TensorFlow, Scikit-learn, XGBoost, or LightGBM models), all you have to do is call `batch_predictor.predict(test_data)`.

For more details see [Ray AIR Predictors](https://docs.ray.io/en/latest/ray-air/predictors.html) and [Ray Serve](https://docs.ray.io/en/latest/serve/getting_started.html) docs.

In [ ]:
import pickle
import numpy as np
import pandas as pd
import pyarrow
import pyarrow.parquet as pq
# Import forecasting libraries. 
import torch
import pytorch_lightning as pl
import pytorch_forecasting as ptf
# Import ray libraries.
import ray
from ray import serve

In [ ]:
# Get location to saved model.
# print(model_dir)
# print(model_path)

##########
# STEP 1. Instantiate a batch predictor from checkpoint.
##########
batch_predictor = ptf.models.TemporalFusionTransformer.load_from_checkpoint(model_path)
print(f"Batch predictor type: {type(batch_predictor)}")


In [ ]:
##########
# STEP 2. Create some test data. 
##########
# Being lazy, pretend the last test data is our out-of-sample test data.
max_prediction_length = FORECAST_CONFIG['forecast_horizon']
new_prediction_data = df.copy()
new_prediction_data["time_idx"] = new_prediction_data["time_idx"] + max_prediction_length

# Convert data from pandas to PyTorch tensors.
print(f"Input data type: {type(df)}")
_, _, test_loader = convert_pandas_pytorch_timeseriesdata(
    new_prediction_data, FORECAST_CONFIG
)
# Checkpoint a custom object.
pickle_dir = model_dir + "test_loader.pickle"
with open(pickle_dir, 'wb') as handle:
    pickle.dump(validation_loader, handle)
    
# Restore a test loader from pickle file.
filename = model_dir + "test_loader.pickle"
with open(filename, 'rb') as handle:
    test_loader = pickle.load(handle)

print(f"Output data type: {type(test_loader)}")

In [ ]:
##########
# STEP 3. Define a Ray Serve deployment class.
##########
@serve.deployment
class ForecastPredictor:
    def __init__(self, predictor, test_data):
        self.predictor = predictor
        self.test_data = test_data
        
    def predict(self):
        raw_predictions, x = \
          self.predictor.predict(self.test_data, mode="raw", return_x=True)
        return x, raw_predictions

    def __call__(self):
        x, raw_predictions = self.predict()
        return [x, raw_predictions]

##########
# STEP 4. Deploy the predictor.
##########
# Bind arguments to the Class constructor.
my_first_deployment = ForecastPredictor.bind(
    predictor=batch_predictor,
    test_data=test_loader)

##########
# STEP 5. Query the deployment and get the result.
##########
# Get handle from serve.run().
handle = serve.run(my_first_deployment)

# ray.get() the results from the handle.
ray_return = ray.get(handle.remote())
new_x = ray_return[0]
new_pred = ray_return[1]

In [ ]:
# Plot some predictions from deployed predictor.
for idx in plot_locations:
    batch_predictor.plot_prediction(new_x, new_pred, idx=idx, show_future_observed=True)